In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 40.8 MB/s 
     |████████████████████████████████| 61 kB 434 kB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
     |████████████████████████████████| 596 kB 51.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import DPRContextEncoderTokenizer as dct
from transformers import DPRContextEncoder as dce

In [ ]:
from transformers import DPRQuestionEncoderTokenizer as dqt
from transformers import DPRQuestionEncoder as dqe


In [ ]:
# firstly, initialize tokenizers and models for both our context model and question model

cntx_model = dce.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
cntx_tokenizer = dct.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')


quest_model = dqe.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

quest_tokenizer = dqt.from_pretrained('facebook/dpr-question_encoder-single-nq-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [ ]:
#Here i want to create some questions and answers to the questions. Take time here to create this, you basically need to ask a question and get a response. This will show why and how the dpr is different from sentence
#transformers. So if i ask a question "At what time will you be coming back tommorow ?" i would not be expecting an anscwer such as "I will  be coming back tommorow" or "We will see each other tommorrow"
# see it dosent answer the initial question. Feel free to add more here

question = [
            "How old is your cat",
            "will you be getting married",
            "what is your favorite movie yet",
            "when is the Queens birthday celebration"
]

context = [
           "Of all my cousins, Ahmed is my favorite",
           "I havent been to Paris yet",
           "My cat is 3 years old",
           "I have written a bestselling book",
           "I wont be getting a cat yet",
           "the queens birthday is in July",
           "i will be getting married",
           "My best movie is Lord of the rings",

]

In [ ]:
cntx_token = cntx_tokenizer(context, max_length=256, padding ='max_length',return_tensors='pt')
#here we create our context embeddings
cntx_embedding = cntx_model(**cntx_token)

quest_token = quest_tokenizer(question, max_length=256, padding ='max_length',return_tensors='pt')
quest_embedding = quest_model(**quest_token)

In [ ]:
quest_embedding.keys()

odict_keys(['pooler_output'])

In [ ]:
quest_embedding.pooler_output.shape

torch.Size([4, 768])

In [ ]:
#This is the requirement for running this example. if you already have this module feel free to skip this line of code

#!pip install sentence-transformers

#compare our query embeddings against our context embeddings to see which are the most similar with cosine similarity

import torch
from sentence_transformers.util import cos_sim as cs

for i , quest_vect in enumerate(quest_embedding.pooler_output):
  probs = cs(quest_vect, cntx_embedding.pooler_output)
  argmax = torch.argmax(probs)
  print(question[i])
  print(context[argmax])
  print('---')

How old is your cat
My cat is 3 years old
---
will you be getting married
i will be getting married
---
what is your favorite movie yet
My best movie is Lord of the rings
---
when is the Queens birthday celebration
the queens birthday is in July
---
